In [1]:
import pandas as pd
#import matplotlib.ticker as mtick
import numpy as np
from arcgis.gis import *
gis = GIS()

In [2]:
#Layer name for K-12 private enrollment data
lyrK12 = 'Schools_PreKto12_Private'
lyrTAZ = 'USTM_TAZ_2021_09_22'

#Column Names
cnLevel    = 'LEVEL2'
cnName     = 'PINST'
cnG00      = 'P160'            #kindergarten as grade 0
cnG01      = 'P180'
cnG02      = 'P200'
cnG03      = 'P210'
cnG04      = 'P220'
cnG05      = 'P230'
cnG06      = 'P240'
cnG07      = 'P250'
cnG08      = 'P260'
cnG09      = 'P270'
cnG10      = 'P280'
cnG11      = 'P290'
cnG12      = 'P300'
cnTot      = 'P305'
cnID       = 'PPIN'

#enrollment levels - grade groups 
elElem = 'Enrol_Elem'
elMidl = 'Enrol_Midl'
elHigh = 'Enrol_High'

#school levels - combo of original data and new levels
slKth8 = 1
slHigh = 2
slOthr = 3

#name of joined layers created later
lyrK12TAZ = 'Schools_PreKto12_Private_withTAZ'
lyrTAZEnrol = 'USTM_TAZ_2021_09_22_Enrol'

In [3]:
#grade column names array
cnGrades   = [cnG00,cnG01,cnG02,cnG03,cnG04,cnG05,cnG06,cnG07,cnG08,cnG09,cnG10,cnG11,cnG12]

# list of Enrollment levels by Grade and by School_Level
colnames  =  [cnLevel, cnG00 , cnG01 , cnG02 , cnG03 , cnG04 , cnG05 , cnG06 , cnG07 , cnG08 , cnG09 , cnG10 , cnG11 , cnG12 ]
enroldata = [[slHigh , elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh, elHigh],
             [slKth8 , elElem, elElem, elElem, elElem, elElem, elElem, elElem, elMidl, elMidl, elMidl, elMidl, elMidl, elMidl],
             [slOthr , elElem, elElem, elElem, elElem, elElem, elElem, elElem, elMidl, elMidl, elMidl, elHigh, elHigh, elHigh]
            ]
  
# Create the pandas DataFrame
dfEnrollLevels = pd.DataFrame(enroldata, columns = colnames)
print('\nEnrollment Levels by School Level and Grade:')
display(dfEnrollLevels)


Enrollment Levels by School Level and Grade:


,LEVEL2,P160,P180,P200,P210,P220,P230,P240,P250,P260,P270,P280,P290,P300
0,2,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High,Enrol_High
1,1,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Midl,Enrol_Midl,Enrol_Midl,Enrol_Midl,Enrol_Midl,Enrol_Midl
2,3,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Elem,Enrol_Midl,Enrol_Midl,Enrol_Midl,Enrol_High,Enrol_High,Enrol_High


In [4]:
dfK12 = pd.DataFrame.spatial.from_featureclass(lyrK12)

#display data totals before filtering
print ("Total Enrollment: " + "{:,}".format(dfK12[cnTot].sum()))
print ("Total Schools: "    + "{:,}".format(dfK12.shape[0])    )

#created filtered dataset that removes unnecessary rows

#make copy to preserve original data
dfK12_fltr = dfK12.copy()

#only school with enrollment
dfK12_fltr = dfK12_fltr[dfK12_fltr[cnTot] > 0]


Total Enrollment: 14,792
Total Schools: 122


In [5]:
#make copy to preserve pre reclassification
dfK12_fltr_rc = dfK12_fltr.copy()
dfK12_fltr_rc[cnLevel] = dfK12_fltr_rc[cnLevel].astype("int32")

#display enrollment by grade to get understanding of the distribution of enrollment for each School_Level
print("\nPre-Reclassify:")
display(dfK12_fltr.groupby([cnLevel], as_index=False).agg(SchoolCount=(cnID,'size'),G00=(cnG00,'sum'),G01=(cnG01,'sum'),G02=(cnG02,'sum'),G03=(cnG03,'sum'),G04=(cnG04,'sum'),G05=(cnG05,'sum'),G06=(cnG06,'sum'),G07=(cnG07,'sum'),G08=(cnG08,'sum'),G09=(cnG09,'sum'),G10=(cnG10,'sum'),G11=(cnG11,'sum'),G12=(cnG12,'sum'),Tot=(cnTot,'sum')))

#display enrollment by grade to get understanding of the distribution of enrollment for each School_Level
print("\nPost-Reclassify:")
display(dfK12_fltr_rc.groupby([cnLevel], as_index=False).agg(SchoolCount=(cnID,'size'),G00=(cnG00,'sum'),G01=(cnG01,'sum'),G02=(cnG02,'sum'),G03=(cnG03,'sum'),G04=(cnG04,'sum'),G05=(cnG05,'sum'),G06=(cnG06,'sum'),G07=(cnG07,'sum'),G08=(cnG08,'sum'),G09=(cnG09,'sum'),G10=(cnG10,'sum'),G11=(cnG11,'sum'),G12=(cnG12,'sum'),Tot=(cnTot,'sum')))


Pre-Reclassify:


,LEVEL2,SchoolCount,G00,G01,G02,G03,G04,G05,G06,G07,G08,G09,G10,G11,G12,Tot
0,1,58,909,4,510,459,420,394,514,431,452,7,1,1,2,7107
1,2,39,1,0,0,2,1,0,7,80,158,636,696,670,588,2840
2,3,25,284,0,293,278,314,313,331,412,386,426,378,353,365,4845



Post-Reclassify:


,LEVEL2,SchoolCount,G00,G01,G02,G03,G04,G05,G06,G07,G08,G09,G10,G11,G12,Tot
0,1,58,909,4,510,459,420,394,514,431,452,7,1,1,2,7107
1,2,39,1,0,0,2,1,0,7,80,158,636,696,670,588,2840
2,3,25,284,0,293,278,314,313,331,412,386,426,378,353,365,4845


In [6]:
#calculate enrollment by level for all schools

#normalize (reverse-pivot) enrollment by grade for ease of calcs
dfK12_melt = pd.melt(dfK12_fltr_rc, id_vars=[cnID,cnLevel], value_vars=cnGrades)
dfK12_melt = dfK12_melt.rename(columns={'variable':'Grade','value':'Enrollment'})
#display(dfK12_melt)

#create 'lookup table' for Enrollment Level
dfEnrollLevels_melt = pd.melt(dfEnrollLevels, id_vars=[cnLevel], value_vars=cnGrades)
dfEnrollLevels_melt = dfEnrollLevels_melt.rename(columns={'variable':'Grade','value':'EnrollmentLevel'})
#display(dfEnrollLevels_melt)

#join to get enrollment levels
dfK12_enrolllevels = pd.DataFrame.merge(dfK12_melt,dfEnrollLevels_melt,on=(cnLevel,'Grade'))

#group to get subtotals by School, Group, and Enrollment Level
dfK12_enrolllevels = dfK12_enrolllevels.groupby([cnID,'EnrollmentLevel'],as_index=False).agg(Enrollment=('Enrollment','sum'))
dfK12_enrolllevels

,PPIN,EnrollmentLevel,Enrollment
0,1412727,Enrol_Elem,150
1,1412727,Enrol_Midl,45
2,1412738,Enrol_Elem,68
3,1412738,Enrol_Midl,23
4,1412749,Enrol_Elem,103
...,...,...,...
225,AA890947,Enrol_Elem,6
226,AA890947,Enrol_High,26
227,AA890947,Enrol_Midl,13
228,K9306116,Enrol_Elem,54


In [7]:
# Pivot by EnrollmentLevel
dfK12_enrol = dfK12_enrolllevels.pivot(index=cnID, columns='EnrollmentLevel', values='Enrollment')

# Flatten the column names (though it might not be needed anymore with single-level columns)
dfK12_enrol.columns = [str(col) for col in dfK12_enrol.columns]

# Fill missing values and convert to integers
dfK12_enrol = dfK12_enrol.fillna(0).astype(int)

display(dfK12_enrol)
display(dfK12_enrol.sum())
display(dfK12_enrol.sum().sum())

,Enrol_Elem,Enrol_High,Enrol_Midl
PPIN,,,
1412727,150,0,45
1412738,68,0,23
1412749,103,0,48
1412771,0,504,0
1412818,0,171,0
...,...,...,...
A9904321,292,0,89
A9904322,62,0,16
AA001502,394,234,264


Enrol_Elem    5023
Enrol_High    3935
Enrol_Midl    2118
dtype: int64

11076

In [8]:
#export school enrollment results to csv
strprivK12File = "results\privk12_enrol.csv"

dfK12_enrol.to_csv(strprivK12File)

#join csv with feature class
arcpy.management.AddJoin(lyrK12, cnID, strprivK12File, cnID,'KEEP_ALL')

#spatial join to TAZ layers
arcpy.analysis.SpatialJoin(lyrK12, lyrTAZ, lyrK12TAZ, join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_COMMON',match_option='WITHIN')

#remove join
arcpy.RemoveJoin_management(lyrK12)

<Result 'Schools_PreKto12_Private'>

In [9]:
#get joined school/TAZ layer in order to aggregate to CO_TAZID
sdfK12TAZ = pd.DataFrame.spatial.from_featureclass(lyrK12TAZ)
dfK12TAZ = sdfK12TAZ[[cnName,'CO_TAZID',elElem,elMidl,elHigh]]
#dfK12TAZ = sdfK12TAZ.fillna(0)
#display(dfK12TAZ.columns)
display(dfK12TAZ)

,PINST,CO_TAZID,Enrol_Elem,Enrol_Midl,Enrol_High
0,ST VINCENT DE PAUL PARISH SCHOOL,350778,150,45,0
1,KEARNS-SAINT ANN SCHOOL,350542,68,23,0
2,OUR LADY OF LOURDES CATHOLIC SCHOOL,350257,103,48,0
3,JUDGE MEMORIAL CATHOLIC HIGH SCHOOL,350257,0,0,504
4,ST JOSEPH CATHOLIC HIGH SCHOOL,570251,0,0,171
...,...,...,...,...,...
117,THE MADELEINE CHOIR SCHOOL,350134,292,89,0
118,PRINCE OF PEACE EVANGELICAL LUTHERAN SCHOOL,350676,62,16,0
119,THE WATERFORD SCHOOL,350988,394,264,234
120,UNIVERSITY ACADEMY,350197,6,13,26


In [10]:
#aggregate by CO_TAZID
dfTAZSummary = dfK12TAZ.groupby(['CO_TAZID']).agg(sum)
dfTAZSummary[dfTAZSummary.select_dtypes(include='number').columns] = \
    dfTAZSummary.select_dtypes(include='number').astype(int)

#remove rows with all zeros
dfTAZSummary = dfTAZSummary.loc[~(dfTAZSummary==0).all(axis=1)]

dfTAZSummary

,PINST,Enrol_Elem,Enrol_Midl,Enrol_High
CO_TAZID,,,,
17126,TURN ABOUT RANCH,0,4,8
21034,RENEWED HOPE RANCH ACADEMY,0,0,28
21122,HAVENWOOD SCHOOLS,0,0,36
21173,TURNING POINT SCHOOL,0,10,10
21215,MOUNTAIN SPRINGS PREPARATORY ACADEMY,0,0,36
...,...,...,...,...
570301,DEAMUDE ADVENTIST CHRISTIAN SCHOOLDEAMUDE ADVE...,13,0,0
570390,NEW DISCOVERY MONTESSORI SCHOOL,6,0,0
570403,WASATCH CHRISTIAN SCHOOL,41,5,0


In [11]:
#export CO_TAZID enrollment to csv

sFilename = 'results\\privK12_Enrollment_' + lyrTAZ + '.csv'

dfTAZSummary.to_csv(sFilename)
print('CSV Exported to: ' + sFilename)

CSV Exported to: results\privK12_Enrollment_USTM_TAZ_2021_09_22.csv


In [12]:
#spatial join to TAZ layer
arcpy.management.AddJoin(lyrTAZ,'CO_TAZID','results\\privK12_Enrollment_' + lyrTAZ + '.csv','CO_TAZID','KEEP_ALL')

#perform code while joined
arcpy.management.CopyFeatures(lyrTAZ, 'results\\privtaz_with_enrollment.shp')

#remove join so that code can be rerun
arcpy.RemoveJoin_management(lyrTAZ)

<Result 'USTM_TAZ_2021_09_22'>